In [1]:
# Подключаем nltk
import nltk
nltk.download('all')
# Подключаем библиотеку для работы с регулярными выражениями
from nltk import word_tokenize
import re
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn import feature_extraction
from scipy import sparse
from sklearn import linear_model
from sklearn.utils import shuffle

import random

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat

[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity 

[nltk_data]    |   Package mte_teip5 is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package perluniprops to C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package perluniprops is already up-to-date!
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\Егор
[nltk_data]    |     Самохин\AppData\Roaming\nltk_data...
[nltk_data]    |   Package nonbreaking_prefixes is already up-to-date!


In [2]:
# Тематика - магазин новогодних игрушек
#1.Приветствие  +
#2.Прощание  +
#3.Благодарность +
#4.Новогодние игрушки +
#5.Герлянды + 
#6.Мишура +
#7.Доствка + 
#8.Свечи (подсвечники) + 
#9.Ёлки (какой тип дерева) +
#10.Декор (гномики, звёздочки и т.д.) +
#11.Способы оплаты + 
#12.Адрес и время работы магазина +

intents = [#намерения 
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hey",
        "How are you",
        "Is anyone there?",
        "Hello",
        "Good day"
      ],
      "responses": [
        "Hey :-)",
        "Hello, thanks for visiting",
        "Hi there, what can I do for you?",
        "Hi there, how can I help?"
      ]
    },
    
    {
      "tag": "goodbye",
      "patterns": ["Bye", "See you later", "Goodbye", "bye"],
      "responses": [
        "See you later, thanks for visiting",
        "Have a nice day",
        "Bye! Come back again soon."
      ]
    },
    
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "That's helpful", "Thank's a lot!"],
      "responses": ["Happy to help!", "Any time!", "My pleasure"]
    },
    
    {
      "tag": "payments",
      "patterns": [
        "Do you take credit cards?",
        "Do you accept Mastercard?",
        "Can I pay with Paypal?",
        "Are you cash only?",
        "How can I pay?"
      ],
      "responses": [
        "We accept VISA, Mastercard and Paypal",
        "We accept most major credit cards, and Paypal",
        "We accept cash at our store"
      ]
    },
    
    {
      "tag": "delivery",
      "patterns": [
        "How long does delivery take?",
        "How long does shipping take?",
        "When do I get my delivery?",
        "How long I need wait?"
      ],
      "responses": [
        "Delivery takes 2-4 days",
        "Shipping takes 2-4 days"
      ]
    },
    
    {
      "tag": "address",
      "patterns": [
        "Where is your store located?",
        "How to get to your store?",
        "What time is your store open?",
        "Which metro station is closer to you?"  
      ],
      "responses": [
        "Our store is located on Molodezhnaya street, 35, office 7.",
        "Our store is open from 9 am to 8 pm",
        "The nearest metro station is Molodyozhnaya"
      ]
    },
    
    {
        "tag": "Christmas_toys",
        "patterns": [
            "What kind of Christmas toys do you have?",
            "Do you have toys in the form of Santa Claus?",
            "Do you have golden balls?",
            "Do you have Christmas bells?"  
      ],
        "responses": [
            "We have a lot of Christmas toys in stock",
            "There are toys in the warehouse in the form of Santa Claus and Snow Maiden",
            "Balls of various colors are sold in packs of 9",
            "A set of small metal bells will decorate your tree."
      ]
        
    },
    
    {
        "tag": "garland",
        "patterns": [
            "Do you have a Christmas tree garland?",
            "How long is your garland?",
            "Can your garland be controlled remotely?",
            "How many modes of operation are there in your garland?",
            "Can I order a small garland from you?"
           
      ],
        "responses": [
            "We have both small and large garlands in stock",
            "Garlands can work automatic",
            "Some of the garland models are controlled using the app on your smartphone"
      ]
    },
    
    {
        "tag": "tinsel",
        "patterns": [
            "Tinsel in what colors do you have?",
            "What is the average length of your tinsel?",
            "What material is your tinsel made of?"
           
      ],
        "responses": [
            "Tinsel of various colors is available in our store",
            "The average for tinsel is 200 centimeters",
            "Tinsel is usually made from polyethylene terephthalate."
      ]
    },
    
    {
        "tag": "candlesticks",
        "patterns": [
            "Do you have classic candles?",
            "Do you have any scented candles?",
            "Do you sell a candlestick in the shape of a lantern?",
            "Do you have in stock decorative candles in the shape of a gnome?"
           
      ],
        "responses": [
            "There are many candles in stock in our store.",
            "You can see candlesticks of various shapes and sizes in our store",
            "You can find decorative candles in the shape of a gnome and a monkey in our store."
      ]
    },
    
    {
        "tag": "tree",
        "patterns": [
            "Do you sell natural or artificial Christmas trees?",
            "How tall are your trees?",
            "Is the tree stand included?",
            "What material are your tree branches made of?"
           
      ],
        "responses": [
            "Artificial trees are sold in our store",
            "The average height of the tree is 1.80 meters",
            "The stand is included in the kit along with the tree",
            "Christmas tree branches are made of plastic and rubber."
      ]
    },
    
    {
        "tag": "decor",
        "patterns": [
            "Do you have Christmas socks in your store?",
            "Can I buy gift wrapping paper from you?",
            "Are there any Christmas cards in your store?",
            "Do you sell Christmas hats?"
           
      ],
        "responses": [
            "Our store has socks for gifts in different sizes.",
            "You can buy various paper and packaging for your gifts from us.",
            "Add our signature birthday cards to them",
            "Our store sells caps for both adults and children."
      ]
    }
]

In [3]:
X_train = []
y_train = []

for intent in intents:
    tag = intent['tag']
    for pattern in intent['patterns']:
        X_train.append(pattern)
        y_train.append(tag)

class StemmedCountVectorizer(feature_extraction.text.CountVectorizer):
    def build_analyzer(self):
        stemmer = nltk.stem.PorterStemmer()
        analyzer = super().build_analyzer()

        def stemming_step(doc):
            stemmed = [stemmer.stem(w) for w in analyzer(doc)]
            stemmed = [token for token in stemmed if re.match(r'\w+', token)] # Отфильтровываем знаки препинания
            return stemmed

        return stemming_step


vectorizer = StemmedCountVectorizer(lowercase=True, tokenizer=nltk.word_tokenize, stop_words=['the', 'a'])
vectorizer.fit(X_train) 

labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(list(set(y_train)))

c:\python\lib\site-packages\sklearn\feature_extraction\text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


LabelEncoder()

In [4]:
len(vectorizer.get_feature_names())

106

In [5]:
X_train = vectorizer.transform(X_train)
y_train = labelEncoder.transform(y_train)

In [6]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [7]:
num_classes = len(labelEncoder.classes_)
input_shape = (len(vectorizer.get_feature_names()), )

In [8]:
X_train = X_train.toarray()
y_train = to_categorical(y_train, num_classes=num_classes)

In [9]:
model = Sequential()
model.add(Dense(40, input_shape=input_shape, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=100, batch_size=128, verbose=1, validation_split=0.0)

Epoch 1/100
1/1 [==============================] - 1s 769ms/step - loss: 2.4851 - accuracy: 0.0588
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 2.4754 - accuracy: 0.0588
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 2.4659 - accuracy: 0.0588
Epoch 4/100
1/1 [==============================] - 0s 4ms/step - loss: 2.4564 - accuracy: 0.0980
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 2.4470 - accuracy: 0.1373
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 2.4378 - accuracy: 0.1569
Epoch 7/100
1/1 [==============================] - 0s 7ms/step - loss: 2.4286 - accuracy: 0.1569
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 2.4195 - accuracy: 0.1961
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 2.4107 - accuracy: 0.1961
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 2.4020 - accuracy: 0.1961
Epoch 11/100
1/1 [=======

1/1 [==============================] - 0s 5ms/step - loss: 1.2897 - accuracy: 0.8824
Epoch 85/100
1/1 [==============================] - 0s 4ms/step - loss: 1.2693 - accuracy: 0.8824
Epoch 86/100
1/1 [==============================] - 0s 5ms/step - loss: 1.2487 - accuracy: 0.8824
Epoch 87/100
1/1 [==============================] - 0s 3ms/step - loss: 1.2280 - accuracy: 0.8824
Epoch 88/100
1/1 [==============================] - 0s 5ms/step - loss: 1.2074 - accuracy: 0.8824
Epoch 89/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1867 - accuracy: 0.8824
Epoch 90/100
1/1 [==============================] - 0s 5ms/step - loss: 1.1660 - accuracy: 0.8824
Epoch 91/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1454 - accuracy: 0.8824
Epoch 92/100
1/1 [==============================] - 0s 4ms/step - loss: 1.1248 - accuracy: 0.8824
Epoch 93/100
1/1 [==============================] - 0s 6ms/step - loss: 1.1042 - accuracy: 0.8824
Epoch 94/100
1/1 [===============

In [12]:
labelEncoder.inverse_transform(
    [ model.predict(vectorizer.transform(['Paying methods'])).reshape(-1).argmax() ]
)

array(['payments'], dtype='<U14')

In [ ]:
while True:
    query = input('> ')
    tag = labelEncoder.inverse_transform(
        [ model.predict(vectorizer.transform([query])).reshape(-1).argmax() ]
    )[0]
    possible_responses = [intent['responses'] for intent in intents if intent['tag'] == tag][0]
    print(random.choice(possible_responses))
    if tag == 'goodbye': break

> Hellow
Hi there, how can I help?
> Have you christmas tree?
The average height of the tree is 1.80 meters
> Have you christmas socks?
You can buy various paper and packaging for your gifts from us.
> Do you have candles?
You can find decorative candles in the shape of a gnome and a monkey in our store.
> Do you have in your store santa?
Our store has socks for gifts in different sizes.
> Okey, bye
Hi there, what can I do for you?
> bye
Hi there, what can I do for you?
> Bye
Hi there, how can I help?
